# Initialization

**Install Modules**

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q pyChatGPT
!pip install -q gradio
!pip install gTTS
!pip install colorama==0.4.3

**Import Modules**

In [35]:
import whisper #Speech To Text model
from pyChatGPT import ChatGPT #ChatBot AI built using NLP
import gradio as gr #User Interface
import warnings #Handle Warnings
from gtts import gTTS #Text To Speech model

**Define Variables**

In [36]:
warnings.filterwarnings("ignore")

In [53]:
secret_token = "" # Enter your session token here!

In [38]:
model = whisper.load_model("base")

In [39]:
model.device

device(type='cuda', index=0)

# Transcribing Function

In [80]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    result_text = result.text

    # Pass the generated text to ChatGPT
    chatgpt_api = ChatGPT(secret_token)
    resp = chatgpt_api.send_message(result_text)
    out_result = resp['message']

    # Text to Speech
    gtts_obj = gTTS(text = out_result, lang="en", tld="ca", slow=False)
    gtts_obj.save("/content/gtts.wav")

    return [result_text, out_result, "/content/gtts.wav"]

# Talk with AI - Gradio User Interface

In [82]:
output_user = gr.Textbox(label="User's Input -")
output_AI = gr.Textbox(label="AI's Response -")
output_AI_audio = gr.Audio(label="Listen...")


gr.Interface(
    title = 'Get all your questions answered from me...', 
    fn = transcribe, 
    inputs=[
        gr.inputs.Audio(source="microphone", label="Talk to me, by pressing this button below", type="filepath")
    ],

    outputs=[
        output_user, output_AI, output_AI_audio
    ],
    live=True, share=True).launch()

Hint: Set streaming=True for Audio component to use live streaming.
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>